<a href="https://colab.research.google.com/github/HungPham2002/Semantic-Search-With-BERT/blob/main/Semantic_Search_With_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.4 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=9d8fb913cbaa080ce29dfd15a2c9d45b2049827db7

In [2]:
from datasets import load_dataset

dataset = load_dataset('ms_marco', 'v1.1')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/10047 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/82326 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9650 [00:00<?, ? examples/s]

Dataset ms_marco downloaded and prepared to /root/.cache/huggingface/datasets/ms_marco/v1.1/1.1.0/b6a62715fa5219aea5275dd3556601004cd63945cb63e36e022f77bb3cbbca84. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
subnet = dataset['test']

In [4]:
queries_infos = []
queries = []
corpus = []

for sample in subnet:
  query_type = sample['query_type']
  if query_type != 'entity':
    continue
  query_id = sample['query_id']
  query_str = sample['query']
  passages_dict = sample['passages']
  is_selected_lst = passages_dict['is_selected']
  passage_text_lst = passages_dict['passage_text']
  query_info = {
      'query_id': query_id,
      'query': query_str,
      'relevant_docs': []
  }
  current_len_corpus = len(corpus)
  for idx in range (len(is_selected_lst)):
    if is_selected_lst[idx] == 1:
      doc_idx = current_len_corpus + idx
      query_info['relevant_docs'].append(doc_idx)

  if query_info['relevant_docs'] == []:
    continue

  queries.append(query_str)
  queries_infos.append(query_info)
  corpus += passage_text_lst

In [8]:
corpus[0]

'SUBPHYLUM CHELICERATA, CLASS ARACHNIDA. Spiders. This group contains many familiar organisms, including the spiders, mites, scorpions and ticks. Examine the large spider on the right. Again, notice that there are two body regions, a cephalothorax and an abdomen. On the cephalothorax are two to four pairs of simple eyes.'

In [5]:
import torch
from sentence_transformers import SentenceTransformer

In [7]:
model = SentenceTransformer('all-MiniLM-L6-v2')
corpus_embeddings = model.encode(corpus, convert_to_tensor = True)

In [9]:
corpus_embeddings.shape

torch.Size([7303, 384])

Custom search function


In [10]:
from sentence_transformers import util

def similarity (query_embeddings, corpus_embeddings):
  return util.cos_sim(query_embeddings, corpus_embeddings)[0]

In [11]:
def ranking(query, top_k=10):
  query_embeddings = model.encode(
      query,
      convert_to_tensor=True
  )
  cos_scores = similarity(
      query_embeddings,
      corpus_embeddings
  )
  top_results = torch.topk(cos_scores, k = top_k)

  return top_results

In [13]:
custom_queries = ['what condition leads to rain', 'ingredients of diphtheria tetanus vaccine']

top_k = min(5, len(corpus))
for query in custom_queries:
  top_results = ranking(query, top_k)

  print("Query:", query)
  print("\n==================")
  print(f"Top {top_k} most similar sentences in corpus:\n")

  for idx, (score, doc_idx) in enumerate(
      zip(top_results[0], top_results[1])
  ):
    print(f'Document rank {idx + 1}:')
    print(corpus[doc_idx], f'\n(Score: {score: .4f})', '\n')

Query: what condition leads to rain

Top 5 most similar sentences in corpus:

Document rank 1:
Rain is liquid water in the form of droplets that have condensed from atmospheric water vapor and then precipitated —that is, become heavy enough to fall under gravity. Rain is a major component of the water cycle and is responsible for depositing most of the fresh water on the Earth. It provides suitable conditions for many types of ecosystems, as well as water for hydroelectric power plants and crop irrigation. When air turbulence occurs, water droplets collide, producing larger droplets. As these larger water droplets descend, coalescence continues, so that drops become heavy enough to overcome air resistance and fall as rain. 
(Score:  0.6689) 

Document rank 2:
SectionsLatest Update: 07/21/97 Rain and Hail Atmospheric conditions that lead to the development of rain and hail. Freezing Rain A detailed look at freezing rain, associated dangers and the conditions that lead to its development